<a href="https://colab.research.google.com/github/neryguilherme/thegamelibrary/blob/main/Classifica%C3%A7%C3%A3o_Mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

# Carregar os dados
banco = pd.read_parquet('/content/games.parquet')

# Reduzir o dataset para 1000 amostras
#if len(banco) > 10000:
#    banco = banco.sample(n=35000, random_state=42)

banco

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",None,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71711,2159120,SUR5,"Apr 1, 2023",0 - 0,0,0,0.00,0,"Innovative Gameplay SUR5 brings a brand new, f...","['English', 'Polish', 'Spanish - Latin America']",...,0,0,0,Katharsis,Katharsis,"Single-player,Multi-player,PvP,Online PvP,Stea...","Free to Play,Indie,Massively Multiplayer,Strat...",None,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71712,2125160,Prison Life 2,"Mar 30, 2023",0 - 20000,6,0,10.79,0,Core Features Real players only! All Guards an...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,AO Games,AO Games,"Single-player,Multi-player,PvP,Online PvP,Co-o...","Casual,Indie,RPG,Simulation,Strategy","First-Person,Detective,Social Deduction,Multip...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71713,2318860,Architecture Zeitgeist,"Mar 27, 2023",0 - 20000,0,0,5.99,0,Architecture Zeitgeist is a game created with ...,"['English', 'Japanese', 'Portuguese - Brazil',...",...,0,0,0,Matheus Rudo,Rudo Studios,Single-player,"Casual,Simulation,Early Access","Casual,Simulation,Puzzle,Exploration,Immersive...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71714,2180330,Girl's Way to Survive 2: Bloody Museum,"Apr 2, 2023",0 - 0,0,0,2.39,0,"Development progress: prologue map (finished),...",['Simplified Chinese'],...,0,0,0,绅士游戏工坊,绅士游戏工坊,Single-player,"Action,Casual,Indie,RPG,Early Access",None,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [ ]:
# Preenchimento de valores faltantes
for col in banco.columns:
    if banco[col].dtype == 'object':
        banco[col] = banco[col].fillna(banco[col].mode()[0])  # Preenche com a moda
    elif pd.api.types.is_numeric_dtype(banco[col]):
        banco[col] = banco[col].fillna(banco[col].median())  # Preenche com a mediana
banco


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy","Indie,Casual",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71711,2159120,SUR5,"Apr 1, 2023",0 - 0,0,0,0.00,0,"Innovative Gameplay SUR5 brings a brand new, f...","['English', 'Polish', 'Spanish - Latin America']",...,0,0,0,Katharsis,Katharsis,"Single-player,Multi-player,PvP,Online PvP,Stea...","Free to Play,Indie,Massively Multiplayer,Strat...","Indie,Casual",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71712,2125160,Prison Life 2,"Mar 30, 2023",0 - 20000,6,0,10.79,0,Core Features Real players only! All Guards an...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,AO Games,AO Games,"Single-player,Multi-player,PvP,Online PvP,Co-o...","Casual,Indie,RPG,Simulation,Strategy","First-Person,Detective,Social Deduction,Multip...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71713,2318860,Architecture Zeitgeist,"Mar 27, 2023",0 - 20000,0,0,5.99,0,Architecture Zeitgeist is a game created with ...,"['English', 'Japanese', 'Portuguese - Brazil',...",...,0,0,0,Matheus Rudo,Rudo Studios,Single-player,"Casual,Simulation,Early Access","Casual,Simulation,Puzzle,Exploration,Immersive...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
71714,2180330,Girl's Way to Survive 2: Bloody Museum,"Apr 2, 2023",0 - 0,0,0,2.39,0,"Development progress: prologue map (finished),...",['Simplified Chinese'],...,0,0,0,绅士游戏工坊,绅士游戏工坊,Single-player,"Action,Casual,Indie,RPG,Early Access","Indie,Casual",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [ ]:
# Separar a coluna 'Name'
if 'Name' in banco.columns:
    name_column = banco['Name']
    x_column = banco.drop(columns=['Name'])
else:
    raise ValueError("A coluna 'Name' não foi encontrada no dataset.")

In [ ]:
# prompt: aplique o label encoder nos atributos

# Aplicar Label Encoding aos atributos categóricos
label_encoders = {}
for column in x_column.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    x_column[column] = le.fit_transform(x_column[column])
    label_encoders[column] = le


In [ ]:

# Escalonar os dados numéricos
scaler = StandardScaler()
numerical_cols = x_column.select_dtypes(include=['number']).columns
x_column[numerical_cols] = scaler.fit_transform(x_column[numerical_cols])
x_column


,AppID,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,-1.970885,1.248325,-0.447085,-0.024282,-0.145418,1.153087,-0.04121,-0.550053,0.568330,0.537784,...,-0.057575,-0.065114,-0.056889,0.451959,0.474674,0.820258,0.787881,0.432102,-0.042294,0.935097
1,-0.909117,1.149448,-0.447085,-0.024282,-0.145418,-0.562958,-0.04121,0.780267,-1.664751,0.537784,...,-0.057575,-0.065114,-0.056889,0.725161,1.419140,0.976020,-0.524940,0.266444,1.048669,-1.141058
2,0.892161,0.908487,-0.447085,-0.024282,-0.145418,-0.201685,-0.04121,-0.086133,-0.159840,0.537784,...,-0.057575,-0.065114,-0.056889,-1.200474,-1.159060,-1.047220,-1.107426,0.351616,-0.470853,0.724920
3,0.261606,-0.191622,-0.447085,-0.024282,-0.145418,-0.111367,-0.04121,-0.451105,-1.723674,0.537784,...,-0.057575,-0.065114,-0.056889,0.342726,0.366923,-1.018395,-0.182930,-1.932185,-1.107465,0.368991
4,-0.099080,-0.673543,-0.447085,-0.024282,-0.145418,-0.652373,-0.04121,-1.456257,0.498658,0.537784,...,-0.057575,-0.065114,-0.056889,1.282838,1.293725,0.900079,0.064672,0.319556,-1.481597,-0.252769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71711,1.604592,-1.737093,-0.803532,-0.024282,-0.145418,-0.652373,-0.04121,-0.178703,-0.224734,-0.567180,...,-0.057575,-0.065114,-0.056889,-0.176756,-0.152119,0.385123,1.076452,0.351616,0.121529,1.620044
71712,1.547823,0.464788,-0.447085,-0.023247,-0.145418,0.322160,-0.04121,-0.997746,-2.123391,0.537784,...,-0.057575,-0.065114,-0.056889,-1.612589,-1.560498,-0.196905,0.714847,-0.025298,0.076565,1.771393
71713,1.871617,0.428228,-0.447085,-0.024282,-0.145418,-0.111367,-0.04121,-1.310922,-0.410659,0.537784,...,-0.057575,-0.065114,-0.056889,0.086878,0.743074,-1.047220,0.875165,-0.431148,0.303152,1.781028
71714,1.640047,-1.637385,-0.803532,-0.024282,-0.145418,-0.436513,-0.04121,-0.886193,0.691749,0.537784,...,-0.057575,-0.065114,-0.056889,1.755610,1.771676,-1.047220,-0.824199,0.351616,0.148498,1.614823


In [ ]:
# prompt: Faça a variancia e remova os atributos inuteis e pront as colunas inuteis

# Calcular a variância de cada coluna
variances = x_column.var()

# Definir um limiar de variância (ex: 0.01, que representa 1%)
threshold = 0.01

# Identificar colunas com variância abaixo do limiar
low_variance_cols = variances[variances < threshold].index.tolist()

# Remover as colunas de baixa variância
#x_column = x_column.drop(columns=low_variance_cols)

# Imprimir as colunas que foram removidas
#print("Colunas removidas devido à baixa variância:")
#print(low_variance_cols)

# Remover colunas com apenas um valor único
#unique_value_cols = [col for col in x_column.columns if x_column[col].nunique() == 1]
#x_column = x_column.drop(columns=unique_value_cols)
#print("Colunas removidas com valor único:")
#print(unique_value_cols)
x_column


,AppID,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,-1.970885,1.248325,-0.447085,-0.024282,-0.145418,1.153087,-0.04121,-0.550053,0.568330,0.537784,...,-0.057575,-0.065114,-0.056889,0.451959,0.474674,0.820258,0.787881,0.432102,-0.042294,0.935097
1,-0.909117,1.149448,-0.447085,-0.024282,-0.145418,-0.562958,-0.04121,0.780267,-1.664751,0.537784,...,-0.057575,-0.065114,-0.056889,0.725161,1.419140,0.976020,-0.524940,0.266444,1.048669,-1.141058
2,0.892161,0.908487,-0.447085,-0.024282,-0.145418,-0.201685,-0.04121,-0.086133,-0.159840,0.537784,...,-0.057575,-0.065114,-0.056889,-1.200474,-1.159060,-1.047220,-1.107426,0.351616,-0.470853,0.724920
3,0.261606,-0.191622,-0.447085,-0.024282,-0.145418,-0.111367,-0.04121,-0.451105,-1.723674,0.537784,...,-0.057575,-0.065114,-0.056889,0.342726,0.366923,-1.018395,-0.182930,-1.932185,-1.107465,0.368991
4,-0.099080,-0.673543,-0.447085,-0.024282,-0.145418,-0.652373,-0.04121,-1.456257,0.498658,0.537784,...,-0.057575,-0.065114,-0.056889,1.282838,1.293725,0.900079,0.064672,0.319556,-1.481597,-0.252769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71711,1.604592,-1.737093,-0.803532,-0.024282,-0.145418,-0.652373,-0.04121,-0.178703,-0.224734,-0.567180,...,-0.057575,-0.065114,-0.056889,-0.176756,-0.152119,0.385123,1.076452,0.351616,0.121529,1.620044
71712,1.547823,0.464788,-0.447085,-0.023247,-0.145418,0.322160,-0.04121,-0.997746,-2.123391,0.537784,...,-0.057575,-0.065114,-0.056889,-1.612589,-1.560498,-0.196905,0.714847,-0.025298,0.076565,1.771393
71713,1.871617,0.428228,-0.447085,-0.024282,-0.145418,-0.111367,-0.04121,-1.310922,-0.410659,0.537784,...,-0.057575,-0.065114,-0.056889,0.086878,0.743074,-1.047220,0.875165,-0.431148,0.303152,1.781028
71714,1.640047,-1.637385,-0.803532,-0.024282,-0.145418,-0.436513,-0.04121,-0.886193,0.691749,0.537784,...,-0.057575,-0.065114,-0.056889,1.755610,1.771676,-1.047220,-0.824199,0.351616,0.148498,1.614823


In [ ]:
# Selecionar features (ajuste conforme necessário)
#features = ['Tags', 'Categories', 'Price']
#features = [f for f in features if f in x_column.columns]
X = x_column
X.columns

Index(['AppID', 'Release date', 'Estimated owners', 'Peak CCU', 'Required age',
       'Price', 'DLC count', 'About the game', 'Supported languages',
       'Full audio languages', 'Reviews', 'Header image', 'Website',
       'Support url', 'Support email', 'Windows', 'Mac', 'Linux',
       'Metacritic score', 'Metacritic url', 'User score', 'Positive',
       'Negative', 'Score rank', 'Achievements', 'Recommendations', 'Notes',
       'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')

In [ ]:
#y = le.fit_transform(name_column)
k = banco['Genres'] #Definindo k para a coluna Genres
k = le.fit_transform(k)

In [ ]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, k, test_size=0.3)

# Criar e treinar o modelo MLP
model = MLPClassifier(max_iter=100, verbose=True, tol=0.0000100,solver = 'adam',hidden_layer_sizes = (50,50),activation = 'relu',random_state=42)
model.fit(X_train, y_train)


# Fazer previsões e avaliar o modelo
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)
#metricas do teste
accuracy_model = accuracy_score(y_test, y_pred)
precision_score_model = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall_score_model = recall_score(y_test, y_pred, average='weighted', zero_division=0)
f1_score_model = f1_score(y_test, y_pred, average='weighted', zero_division=0)
#metricas do treino
accuracy_train = accuracy_score(y_train, y_pred_train)
precision_score_train = precision_score(y_train, y_pred_train, average='weighted', zero_division=0)
recall_score_train = recall_score(y_train, y_pred_train, average='weighted', zero_division=0)
f1_score_train = f1_score(y_train, y_pred_train, average='weighted', zero_division=0)

#
print(classification_report(y_test, y_pred, zero_division=0))
print("-" * 50)
print(f"Acurácia no conjunto de treinamento: {accuracy_train}")
print(f"Precisão no conjunto de treinamento: {precision_score_train}")
print(f"Recall no conjunto de treinamento: {recall_score_train}")
print(f"F1-score no conjunto de treinamento: {f1_score_train}")
print("-" * 50)
print(f"Acurácia no conjunto de teste: {accuracy_model}")
print(f"Precisão no conjunto de teste: {precision_score_model}")
print(f"Recall no conjunto de teste: {recall_score_model}")
print(f"F1-score no conjunto de teste: {f1_score_model}")

Iteration 1, loss = 5.07098664
Iteration 2, loss = 3.98937181
Iteration 3, loss = 3.63427563
Iteration 4, loss = 3.37311797
Iteration 5, loss = 3.15150852
Iteration 6, loss = 2.95653157
Iteration 7, loss = 2.78842833
Iteration 8, loss = 2.64032733
Iteration 9, loss = 2.52346925
Iteration 10, loss = 2.41691633
Iteration 11, loss = 2.33981815
Iteration 12, loss = 2.26215722
Iteration 13, loss = 2.19379254
Iteration 14, loss = 2.14895134
Iteration 15, loss = 2.09285997
Iteration 16, loss = 2.06636162
Iteration 17, loss = 2.02010682
Iteration 18, loss = 1.98431180
Iteration 19, loss = 1.95306097
Iteration 20, loss = 1.91487067
Iteration 21, loss = 1.87318346
Iteration 22, loss = 1.86034979
Iteration 23, loss = 1.82636231
Iteration 24, loss = 1.81451248
Iteration 25, loss = 1.79230165
Iteration 26, loss = 1.75755600
Iteration 27, loss = 1.75011231
Iteration 28, loss = 1.72130289
Iteration 29, loss = 1.69900169
Iteration 30, loss = 1.67731497
Iteration 31, loss = 1.66474955
Iteration 32, los

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.67      0.33      0.44         6
           1       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           6       0.60      0.89      0.72       565
           7       0.61      0.10      0.17       298
           8       0.25      0.03      0.05        67
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         4
          11       0.00      0.00      0.00         2
          12       0.23      0.68      0.35        34
          13       0.00      0.00      0.00         5
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         2
          16       0.00      0.00      0.00         2
          17       0.00      0.00      0.00         1
          19       0.00      0.00      0.00         1
          20       0.00    